## Quick documentation example

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("lrex").getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [4]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [5]:
lr = LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction")

In [6]:
lrModel = lr.fit(training)

In [7]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [8]:
lrModel.intercept

0.14228558260358093

In [10]:
training_summary = lrModel.summary

In [11]:
training_summary.r2

0.027839179518600154

In [12]:
training_summary.meanSquaredError

103.28843028724194

In [13]:
training_summary.rootMeanSquaredError

10.16309157133015

## Correct way to do

In [14]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [15]:
# using the randomSplit() method, which is available for all data frames
train, test = all_data.randomSplit([0.7, 0.3])

In [16]:
# check how many samples in the training data
train.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 333|
|   mean|-0.10172363055297175|
| stddev|   9.946472795137085|
|    min| -26.736207182601724|
|    max|   27.78383192005107|
+-------+--------------------+



In [17]:
# check how many samples in the test data
test.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                168|
|   mean|  0.967709933891845|
| stddev| 11.013362944782005|
|    min|-28.571478869743427|
|    max| 24.290551295953957|
+-------+-------------------+



In [18]:
# train on the training data
correctModel = lr.fit(train)

In [19]:
test_eval = correctModel.evaluate(test)

In [20]:
test_eval.rootMeanSquaredError

11.178265241572804

## Example of predictions

In [21]:
unlabeled_data = test.select("features")

In [23]:
test_pred = correctModel.transform(unlabeled_data)

## Evaluation of regression 

### Load data

In [24]:
spark = SparkSession.builder.appName("lr_example").getOrCreate()

In [26]:
df = spark.read.csv("Ecommerce_Customers.csv", inferSchema=True, header=True)

In [27]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [28]:
for item in df.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


For this simple task, we focus only on numeric features

In [30]:
df.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [31]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [32]:
# create an assembler object
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'],
                           outputCol="features")

In [33]:
# now transform the data

In [34]:
output = assembler.transform(df)

In [35]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [36]:
# create final data by select features and lable
final_data = output.select(["features", "Yearly Amount Spent"])
final_data.show(5)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
+--------------------+-------------------+
only showing top 5 rows



### Split data

In [37]:
train, test = final_data.randomSplit([0.7, 0.3])

In [38]:
train.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                360|
|   mean|  501.2893572530587|
| stddev|  80.10281306722074|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [39]:
test.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                140|
|   mean| 494.23464655853064|
| stddev|  77.30163123882913|
|    min|  304.1355915788555|
|    max|  725.5848140556806|
+-------+-------------------+



### Training

In [40]:
lr1 = LinearRegression(featuresCol = "features", labelCol="Yearly Amount Spent")

In [41]:
lr1Model = lr1.fit(train)

In [42]:
test_eval = lr1Model.evaluate(test)

In [45]:
test_eval.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  18.54941973937258|
| 3.5231660717735167|
| 3.7138963760997967|
| 0.5335953113865344|
| -4.771399889289796|
|-18.541362187702134|
| 17.836616468112425|
| -4.398139405218046|
|  6.231724801583084|
| -26.96400788065432|
|-1.3186631790878778|
|  2.058923661253459|
| -9.550550261441174|
|  1.246644098473098|
|  8.065196779193968|
|-5.6022413771262904|
|-17.625292397132682|
| -1.635215129559981|
| 12.128984827351417|
| -4.714093540109047|
+-------------------+
only showing top 20 rows



In [46]:
test_eval.rootMeanSquaredError

10.444404994427023

In [47]:
test_eval.r2

0.9816133196600121

### Prediction

In [49]:
unlabeled_dt = test.select("features")
unlabeled_dt.show()

+--------------------+
|            features|
+--------------------+
|[31.3123495994443...|
|[31.3584771924370...|
|[31.3662121671876...|
|[31.3895854806643...|
|[31.4252268808548...|
|[31.5702008293202...|
|[31.6005122003032...|
|[31.6253601348306...|
|[31.6548096756927...|
|[31.6739155032749...|
|[31.7216523605090...|
|[31.7366356860502...|
|[31.8279790554652...|
|[31.8293464559211...|
|[31.8512531286083...|
|[31.8745516945853...|
|[31.9048571310136...|
|[31.9120759292006...|
|[31.9262720263601...|
|[31.9673209478824...|
+--------------------+
only showing top 20 rows



In [50]:
preds = lr1Model.transform(unlabeled_dt)

In [51]:
preds.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[31.3123495994443...|445.04199828856804|
|[31.3584771924370...| 491.6527843777019|
|[31.3662121671876...| 426.8749861803851|
|[31.3895854806643...|409.53601574859636|
|[31.4252268808548...| 535.5381185440517|
|[31.5702008293202...|  564.486854329107|
|[31.6005122003032...| 461.3362350229845|
|[31.6253601348306...|380.73504016214224|
|[31.6548096756927...|469.03169892596543|
|[31.6739155032749...| 502.6890757905355|
|[31.7216523605090...| 349.0955898109605|
|[31.7366356860502...|494.87452259427846|
|[31.8279790554652...| 449.5532978083827|
|[31.8293464559211...| 383.9056938895019|
|[31.8512531286083...| 464.9270498876044|
|[31.8745516945853...| 397.8874856233938|
|[31.9048571310136...| 491.5751498199488|
|[31.9120759292006...| 389.1699314352677|
|[31.9262720263601...|  380.075948616975|
|[31.9673209478824...| 450.4639347797613|
+--------------------+------------